In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [ ]:
data = pd.read_csv('/kaggle/input/lifestyle-factors-influencing-osteoporosis/osteoporosis.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isna().sum() 

In [ ]:
# Xử lý dữ liệu None

# Xử lý dữ liệu None cho cột Alcohol Consumption
data['Alcohol Consumption'] = data['Alcohol Consumption'].fillna('None')
# Xử lý dữ liệu None cho cột Medications            
data['Medical Conditions'] = data['Medical Conditions'].fillna('None')

# Xử lý dữ liệu None cho cột Medical Conditions
data['Medications'] = data['Medications'].fillna('None')

In [ ]:
# Data visualization 


In [ ]:
# Histogram của cột Age
plt.hist(data['Age'], bins=20, edgecolor='black')  # bins là số lượng thùng, edgecolor tạo đường viền cho cột
plt.title('Phân phối tuổi')
plt.xlabel('Tuổi')
plt.ylabel('Tần số')
plt.show()


In [ ]:
sns.boxplot(x=data['Age'])
plt.title('Boxplot của cột Age')
plt.xlabel('Tuổi')
plt.show()

In [ ]:
# Statistics của age
age = data['Age']
Q1 = age.quantile(0.25)
Q3 = age.quantile(0.75)
IQR = Q3 - Q1

lowerbound = Q1 - 1.5 * IQR
upperbound = Q3 + 1.5 * IQR
print(f"Trung bình: {age.mean()}")
print(f"Trung vị: {age.median()}")
print(f"Mốt: {age.mode()[0]}")
print(f"Phương sai: {age.var()}")
print(f"Độ lệch chuẩn: {age.std()}")
print(f"Phạm vi: {age.max() - age.min()}")
print(f"Min: {age.min()}")
print(f"Min: {age.max()}")
print(f"Q1: {age.quantile(0.25)}")
print(f"Q3: {age.quantile(0.75)}")
print(f"IQR: {age.quantile(0.75) - age.quantile(0.25)}")
print(f"Lowerbound: {lowerbound}")
print(f"Upperbound: {upperbound}")

In [ ]:
# Tính tay
age = data['Age'].tolist()  # Chuyển cột Age thành list

n = len(age)
age_sorted = sorted(age)

# Trung bình (Mean)
mean = sum(age) / n

# Trung vị (Median)
if n % 2 == 0:
    median = (age_sorted[n // 2 - 1] + age_sorted[n // 2]) / 2
else:
    median = age_sorted[n // 2]

# Mốt (Mode)
counts = {}
for x in age:
    counts[x] = counts.get(x, 0) + 1
max_count = max(counts.values())
mode = [k for k, v in counts.items() if v == max_count][0]

# Phương sai (Variance)
variance = sum((x - mean) ** 2 for x in age) / (n - 1)

# Độ lệch chuẩn (Standard Deviation)
std_dev = variance ** 0.5

# Phạm vi (Range)
range_val = max(age) - min(age)

# Q1 và Q3
q1 = age_sorted[n // 4]
q3 = age_sorted[3 * n // 4]

# IQR
iqr = q3 - q1

# Lowerbound và Upperbound
lowerbound = q1 - 1.5 * iqr
upperbound = q3 + 1.5 * iqr

# In kết quả
print(f"Trung bình: {mean}")
print(f"Trung vị: {median}")
print(f"Mốt: {mode} (xuất hiện nhiều nhất)")
print(f"Phương sai: {variance}")
print(f"Độ lệch chuẩn: {std_dev}")
print(f"Phạm vi: {range_val}")
print(f"Q1: {q1}")
print(f"Q3: {q3}")
print(f"IQR: {iqr}")
print(f"Lowerbound: {lowerbound}")
print(f"Upperbound: {upperbound}")

In [ ]:
# Vẽ Pie Chart cho các cột phân loại
categorical_cols = data.select_dtypes(include=['object']).columns  # Lấy các cột phân loại
sns.set_palette('pastel') 
for col in categorical_cols:
    plt.figure(figsize=(6, 6))
    data[col].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title(f'Pie Chart of {col}')
    plt.ylabel('')  # Loại bỏ label trục y để biểu đồ gọn gàng hơn
    plt.show()

In [ ]:
# Phân phối của các cột theo Osteoporosis

categorical_cols = data.select_dtypes(include=['object']).columns

for col in categorical_cols:
    # Tạo DataFrame tiền xử lý
    grouped_data = data.groupby([col, 'Osteoporosis']).size().unstack(fill_value=0)

    # Vẽ biểu đồ bar chart kép
    plt.figure(figsize=(10, 6))
    grouped_data.plot(kind='bar')

    plt.title(f'Phân phối {col} theo Osteoporosis')
    plt.xlabel(col)
    plt.ylabel('Số lượng')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Osteoporosis', labels=['None', 'Osteoporosis'])

    plt.tight_layout()
    plt.show()


In [ ]:
# Vẽ histogram theo tuổi cho người bị bệnh và không
age_osteoporosis = data[data['Osteoporosis'] == 1]['Age']
age_none = data[data['Osteoporosis'] == 0]['Age']

# Vẽ histogram cho Osteoporosis
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(age_osteoporosis, bins=20, edgecolor='black')
plt.title('Age Distribution (Osteoporosis)')
plt.xlabel('Age')
plt.ylabel('Frequency')

# Vẽ histogram cho None
plt.subplot(1, 2, 2)
plt.hist(age_none, bins=20, edgecolor='black')
plt.title('Age Distribution (None)')
plt.xlabel('Age')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Chuản bị dữ liệu

In [ ]:
# Drop ccột Id vì cột này không có giá trị trotrong việc đánh giá dữ liệu
data = data.drop(columns=["Id"], errors='ignore')
data_clean.head()

In [ ]:
# Mã hóa biến phân loại bằng LabelEncoder

label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [ ]:
# Chọn biến đầu vào và biến mục tiêu
X = data.drop(columns=["Osteoporosis"])
y = data["Osteoporosis"]
# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Chuẩn hóa dữ liệu bằng StandardStandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Huấn luyện mô hình Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)

# Dự đoán
y_pred = model.predict(X_test)

# Đánh giá mô hình
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
'''
Accuracy: ~80.36% → Mô hình có độ chính xác khá cao nhưng vẫn còn sai sót (~20%).
Confusion Matrix cho thấy:
160 trường hợp được dự đoán chính xác không bị loãng xương (True Negative)
155 trường hợp được dự đoán chính xác bị loãng xương (True Positive)
44 trường hợp bị loãng xương nhưng dự đoán sai là không bị (False Negative) → Đây là lỗi nghiêm trọng, vì có thể bỏ sót bệnh nhân thực sự có nguy cơ.
33 trường hợp không bị loãng xương nhưng dự đoán sai là bị (False Positive).
'''

In [ ]:
feature_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_[0]})
feature_importance = feature_importance.sort_values(by='Coefficient', ascending=False)
print(feature_importance)


In [ ]:
plt.figure(figsize=(10, 8))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
# luyện các mô hình và đánh giá bằng các metric quan trọng
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else y_pred
    
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_prob)
    }
results_df = pd.DataFrame(results).T
print(results_df)

In [ ]:
# Huấn luyện lại Decision Tree với max_depth=5
dt_selected = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_selected.fit(X_train_selected, y_train)
y_pred_selected = dt_selected.predict(X_test_selected)

# Đánh giá lại Decision Tree sau khi chọn đặc trưng
print("🔹 Decision Tree (Feature Selection) 🔹")
print(f"Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_pred_selected):.4f}")

In [ ]:
adaboost_model = AdaBoostClassifier()
adaboost_model.fit(X_train, y_train)
decision_tree_model=dt_selected


adaboost_importances = adaboost_model.feature_importances_
dt_importances = decision_tree_model.feature_importances_

# Lấy tên các đặc trưng
feature_names = X.columns  # X là tập dữ liệu đầu vào

# Vẽ biểu đồ so sánh
fig, ax = plt.subplots(1, 2, figsize=(14, 6))

# Decision Tree Feature Importance
ax[0].barh(feature_names, dt_importances, color='skyblue')
ax[0].set_title('Decision Tree - Feature Importance')
ax[0].set_xlabel('Importance')

# AdaBoost Feature Importance
ax[1].barh(feature_names, adaboost_importances, color='salmon')
ax[1].set_title('AdaBoost - Feature Importance')
ax[1].set_xlabel('Importance')

plt.tight_layout()
plt.show()

In [ ]:
df_importance = pd.DataFrame({
    'Feature': feature_names,
    'Decision Tree': dt_importances,
    'AdaBoost': adaboost_importances
})

# Sắp xếp theo mức độ quan trọng giảm dần
df_importance = df_importance.sort_values(by='AdaBoost', ascending=False)
print(df_importance)

In [ ]:
# Kiểm định T-test cho Hypothesis Testing
'''
🔹 Giả thuyết:

H₀ (Giả thuyết gốc): Không có sự khác biệt về tuổi trung bình giữa hai nhóm.
H₁ (Giả thuyết đối): Có sự khác biệt về tuổi trung bình giữa hai nhóm.


'''

np.random.seed(42)
n = 100  # Số quan sát mỗi nhóm
age_healthy = np.random.normal(50, 10, n)  # Nhóm không loãng xương
age_osteoporosis = np.random.normal(60, 12, n)  # Nhóm loãng xương

# Kiểm định T-test
t_stat, p_value = stats.ttest_ind(age_healthy, age_osteoporosis)

# Kết quả
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Nhận xét
if p_value < 0.05:
    print("Bác bỏ H₀: Có sự khác biệt đáng kể về tuổi trung bình giữa hai nhóm.")
else:
    print("Không đủ bằng chứng để bác bỏ H₀: Tuổi trung bình hai nhóm không có khác biệt đáng kể.")


In [ ]:
# Xây dựng khoảng tin cậy (Confidence Interval - CI)
# Tính trung bình và độ lệch chuẩn
mean_age = np.mean(age_osteoporosis)
std_age = np.std(age_osteoporosis, ddof=1)
n_samples = len(age_osteoporosis)

# Tính giá trị t-score cho mức tin cậy 95%
confidence_level = 0.95
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=n_samples-1)

# Khoảng tin cậy
margin_of_error = t_critical * (std_age / np.sqrt(n_samples))
lower_bound = mean_age - margin_of_error
upper_bound = mean_age + margin_of_error

# Kết quả
print(f"Khoảng tin cậy 95% cho tuổi trung bình nhóm loãng xương: ({lower_bound:.2f}, {upper_bound:.2f})")


In [ ]:
# Xây dựng khoảng tin cậy (Confidence Interval - CI)
# Tính trung bình và độ lệch chuẩn
mean_age = np.mean(age_healthy)
std_age = np.std(age_healthy, ddof=1)
n_samples = len(age_healthy)

# Tính giá trị t-score cho mức tin cậy 95%
confidence_level = 0.95
t_critical = stats.t.ppf((1 + confidence_level) / 2, df=n_samples-1)

# Khoảng tin cậy
margin_of_error = t_critical * (std_age / np.sqrt(n_samples))
lower_bound = mean_age - margin_of_error
upper_bound = mean_age + margin_of_error

# Kết quả
print(f"Khoảng tin cậy 95% cho tuổi trung bình nhóm không loãng xương: ({lower_bound:.2f}, {upper_bound:.2f})")


In [ ]:
# Point estimation
mean_age_healthy = np.mean(age_healthy)
mean_age_osteoporosis = np.mean(age_osteoporosis)

print(f"Ước lượng tuổi trung bình nhóm không loãng xương: {mean_age_healthy:.2f}")
print(f"Ước lượng tuổi trung bình nhóm loãng xương: {mean_age_osteoporosis:.2f}")
